In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
print("Setup Complete")

Setup Complete


In [2]:
from google.cloud import bigquery
print("Setup Complete")

Setup Complete


In [3]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_crime", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Your code here to find the table name
tables = list(client.list_tables(dataset))

table_ref = dataset_ref.table("crime")
table = client.get_table(table_ref)
client.list_rows(table , max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,10272397,HY460527,2015-10-09 00:00:00+00:00,111XX S KEDZIE AVE,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,True,...,19,74,20,None,None,2015,2016-04-15 11:16:05+00:00,None,None,None
1,5632101,HN417668,2005-10-01 01:00:00+00:00,079XX S WOODLAWN AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,OTHER,False,False,...,8,45,02,None,None,2005,2015-08-17 15:03:40+00:00,None,None,None
2,4946121,HM559625,2005-09-13 12:00:00+00:00,060XX N CENTRAL AVE,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,...,41,12,06,None,None,2005,2015-08-17 15:03:40+00:00,None,None,None
3,2992155,HJ691078,2003-10-13 18:30:00+00:00,032XX S WELLS ST,1780,OFFENSE INVOLVING CHILDREN,OTHER OFFENSE,STREET,False,True,...,11,34,26,None,None,2003,2015-08-17 15:03:40+00:00,None,None,None
4,6018528,HP123615,2004-01-01 00:00:00+00:00,051XX S KEELER AVE,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,False,...,23,57,20,None,None,2004,2015-08-17 15:03:40+00:00,None,None,None


In [4]:
first_query = """
                       SELECT EXTRACT(YEAR FROM date) AS year, 
                              COUNT(1) AS num_crime
                       FROM `bigquery-public-data.chicago_crime.crime`
                       GROUP BY year
                       ORDER BY year
                       """


# Set up the query (cancel the query if it would use too much of 
# your quota)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
first_query_job = client.query(first_query, job_config=safe_config)


# API request - run the query, and return a pandas DataFrame
first_year_result = first_query_job.to_dataframe()


print(first_year_result)

    year  num_crime
0   2001     485782
1   2002     486764
2   2003     475961
3   2004     469396
4   2005     453732
5   2006     448137
6   2007     437040
7   2008     427097
8   2009     392764
9   2010     370395
10  2011     351875
11  2012     336129
12  2013     307283
13  2014     275527
14  2015     264432
15  2016     269414
16  2017     268646
17  2018     268190
18  2019     260022
19  2020     139391


In [5]:
tORf_query = """ SELECT EXTRACT(YEAR FROM date) AS year , arrest
FROM `bigquery-public-data.chicago_crime.crime` 
WHERE arrest = True AND year = 2010
              """


# Set up the query (cancel the query if it would use too much of 
# your quota)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
tORf_query_job = client.query(tORf_query, job_config=safe_config)


# API request - run the query, and return a pandas DataFrame
tORf_query_result = tORf_query_job.to_dataframe()


tORf_query_result.head()

,year,arrest
0,2010,True
1,2010,True
2,2010,True
3,2010,True
4,2010,True


In [6]:
f_query = """ 
                       SELECT year, sum(CASE WHEN arrest = False THEN 1 ELSE 0 END) as arrest_False
                            FROM `bigquery-public-data.chicago_crime.crime`
                            where year between 2000 and 2020
                            GROUP BY year
                            order by year
                         
              """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
f_query_job = client.query(f_query, job_config=safe_config)


f_query_result = f_query_job.to_dataframe()


print(f_query_result)

    year  arrest_False
0   2001        343872
1   2002        345209
2   2003        334384
3   2004        324706
4   2005        312829
5   2006        312742
6   2007        305178
7   2008        317123
8   2009        281978
9   2010        269892
10  2011        255616
11  2012        245514
12  2013        220793
13  2014        195947
14  2015        194460
15  2016        216472
16  2017        216095
17  2018        214462
18  2019        204362
19  2020        116736


In [7]:
t_query = """ 
                       SELECT year, sum(CASE WHEN arrest = True THEN 1 ELSE 0 END) as arrest_True
                            FROM `bigquery-public-data.chicago_crime.crime`
                            where year between 2000 and 2020
                            GROUP BY year
                            order by year
                         
              """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
t_query_job = client.query(t_query, job_config=safe_config)


t_query_result = t_query_job.to_dataframe()


print(t_query_result)

    year  arrest_True
0   2001       141910
1   2002       141555
2   2003       141577
3   2004       144690
4   2005       140903
5   2006       135395
6   2007       131862
7   2008       109974
8   2009       110786
9   2010       100503
10  2011        96259
11  2012        90615
12  2013        86490
13  2014        79580
14  2015        69972
15  2016        52942
16  2017        52551
17  2018        53728
18  2019        55660
19  2020        22655


In [10]:
tORf_query_result.set_index('year').plot()

TypeError: no numeric data to plot